In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging

logging.basicConfig(level = logging.INFO) 
# Change this to logging.DEBUG pr INFO for more/less detailing debugging information.
# logging.DEBUG(level = logging.DEBUG) 

In [4]:
import os
import sys, pprint

os.getcwd()
pprint.pprint(sys.path)

['/Users/mauf/Git/iot-ingester-deployment/dockerfiles/worker/iot-ingester/notebooks/new_action',
 '/Users/mauf/.pyenv/versions/3.10.8/lib/python310.zip',
 '/Users/mauf/.pyenv/versions/3.10.8/lib/python3.10',
 '/Users/mauf/.pyenv/versions/3.10.8/lib/python3.10/lib-dynload',
 '',
 '/Users/mauf/Git/iot-ingester-deployment/.venv/lib/python3.10/site-packages',
 '/Users/mauf/Git/iot-ingester-deployment/dockerfiles/worker/iot-ingester/src']


# Load in the config files to create iot-ingester actions

In [8]:
from pathlib import Path
from obsproc.core.config_parser import parse_config
from dataclasses import asdict

config_file = Path("config/config.yaml")
config = parse_config(config_file)

# Construct the pipeline from the config
pipeline = {name : getattr(config, name) for name in config.pipeline}
print(f"Global Config:")
for k, v in asdict(config.global_config).items():
    if k == "canonical_variables": v = "..."
    print(f"    {k} : {v}")
    
for name, stage in pipeline.items():
    print(f"\nStage: {name.capitalize()}")
    for pipe in stage:
        print(f"    {str(pipe)}")

AttributeError: 'NoneType' object has no attribute 'strip'

In [6]:
from obsproc.core.bases import FinishMessage, FileMessage, MetaData
from obsproc.aggregators import TimeAggregator

# Pull out the parser object that matches this message
message = FileMessage(
    metadata=MetaData(state='big_file',
        source='sensor_community',
        filepath=Path('../sensor.community/data/inputs/sensor_community/2023-08/2023-08_sds011.csv').resolve())) # May need to download this file manually

actions = [s for stage in list(pipeline.values())[1:] for s in stage]

message_history = []
for _ in range(10):
    display(message)
    message_history.append(message)
    matching = [action for action in actions if action.matches(message)]
    
    if not matching: 
        print("No more matches, the message is fully processed!")
        break

    print("That messages matches with this/these action(s): \n\n", "\n".join(str(a) for a in matching))
    action = matching[0]
    print("\nHere's the first output message from passing the previous message to the first matching action: \n")

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))

NameError: name 'pipeline' is not defined

In [9]:
# Pull out the parser object that matches this message
message = FileMessage(
    metadata=MetaData(state='big_file',
        source='sensor_community',
        filepath=Path('../sensor.community/data/inputs/sensor_community/2023-08/2023-08_sds011.csv').resolve())) # May need to download this file manually

message_history = []
for _ in range(4):
    message_history.append(message)
    matching = [action for action in actions if action.matches(message)]
    
    if not matching: 
        break

    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))

message

TabularMessage(state = parsed, src = sensor_community, obs = P1): Time is not in UTC!
TabularMessage(state = parsed, src = sensor_community, obs = P1): Time is not in UTC!
TabularMessage(state = parsed, src = sensor_community, obs = P1): Time is not in UTC!


state,quality_controlled
source,sensor_community
observation_variable,P1
time_slice,2023-08-01 00:00
name,FileMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVChunker
name,TabularMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVParser
name,TabularMessage
